# Day 7: Camel Cards
The basis for today's code is the ranking of types of the cards on a hand as well as the ranking of the cards.

The ranking of the types of hands is done via an enum and the values ordinals. The biggest part of the code was getting the enum value from the given cards, due to the different handling in case 2.

In [ ]:
enum class HandType(val counts: List<Int>) {
    HighCard(listOf(1, 1, 1, 1, 1)),
    OnePair(listOf(1, 1, 1, 2)),
    TwoPair(listOf(1, 2, 2)),
    ThreeOfAKind(listOf(1, 1, 3)),
    FullHouse(listOf(2, 3)),
    FourOfAKind(listOf(1, 4)),
    FiveOfAKind(listOf(5));
    
    companion object {
        fun fromCards1(cards: List<Char>): HandType {
            val cardCounts = cards.groupBy { it }
                .map { (_, c) -> c.size }
                .sorted()
            
            return HandType.values()
                .first { it.counts == cardCounts }
        }
        
        fun fromCards2(cards: List<Char>): HandType {
            val jCount = cards.filter { it == 'J' }.size
            val cardCounts = cards.filterNot { it == 'J' }
                .groupBy { it }
                .map { (_, c) -> c.size }
                .sorted()
            
            val cardCountsWithJ = if (cardCounts.isEmpty())
                listOf(jCount)
            else
                cardCounts.dropLast(1).plus(cardCounts.last() + jCount)
            
            return HandType.values()
                .first { it.counts == cardCountsWithJ }
        }
    }
}

Again, we have a class for interpreting the input. Before part 2, this was a `Comparable`. With part 2 I decided to extract the comparing code into a ...

In [ ]:
data class Hand(val cards: List<Char>, val bidAmount: Long, val type: HandType)

... `Comparator` that is referenced directly in the sorting function and can be configured with the right card ranking.

In [ ]:
class HandComparator(private val cardRanking: List<Char>) : Comparator<Hand> {
    override fun compare(h0: Hand, h1: Hand): Int {
        if (h0.type != h1.type) {
            return h0.type.ordinal - h1.type.ordinal
        }
        for (i in 0 until h0.cards.size) {
            if (h0.cards[i] != h1.cards[i]) {
                return cardRanking.indexOf(h0.cards[i]) - cardRanking.indexOf(h1.cards[i])
            }
        }
        return 0
    }
}

As usual ... read the input.

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val hands = Path("input.txt").readLines()

Both parts of today's puzzle use the same code but with different configuration.

In [ ]:
fun solve(cardRanking: List<Char>, getHandType: (List<Char>) -> HandType): Long {
    return hands.map { 
        val (cards, bidAmount) = it.split(' ')
        Hand(cards.toList(), bidAmount.toLong(), getHandType(cards.toList()))
    }
    .sortedWith(HandComparator(cardRanking))
    .mapIndexed { i, hand -> hand.bidAmount * (i + 1) }
    .sum()
}

## Part 1

In [ ]:
solve("23456789TJQKA".toList(), HandType::fromCards1)

## Part 2

In [ ]:
solve("J23456789TQKA".toList(), HandType::fromCards2)